### Initializing Spark Session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Bronze_JobETL").getOrCreate()

### Defining required variables

In [0]:
import json, requests

API_KEY = "7f3373f399923bd80f8619eb7f51866eed4af252a228066f0dcfe50d5e5c220a"

roles = ["Data Engineer", "Python Developer", "ETL Developer", "Spark Engineer", "Data Analyst"]
location = "India"
all_jobs = []

### Running a Loop over all the Job Roles and fetching data

In [0]:
for role in roles:
    params = {
        "engine": "google_jobs",
        "q": role,
        "location": location,
        "api_key": API_KEY
    }
    res = requests.get("https://serpapi.com/search.json", params=params)
    print("✅Reading LIVE Data from Google Jobs API")
    jobs = res.json().get("jobs_results", [])
                        
    for job in jobs:
        job["search_role"] = role
    all_jobs.extend(jobs)

### Creating a DataFrame from list of dictionaries


In [0]:
# bronze_df = spark.read.json(spark.sparkContext.parallelize([json.dumps(job) for job in all_jobs]))
# Directly accessing the underlying Spark driver JVM using the attribute 'sparkContext' is not supported on serverless compute.
bronze_df = spark.createDataFrame(all_jobs)

#display(bronze_df)

### Saving the Data in Bronze Delta Table

In [0]:
bronze_df.write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable("lakehouse.bronze.jobs_raw")
print("✅Data written to Bronze Layer")